In [1]:
from torch import nn
import importlib

import my_modules.nsclc
import my_modules.nsclc.nsclc_dataset
import my_modules.custom_models.classifier_models
importlib.reload(my_modules.nsclc.nsclc_dataset)
from my_modules.nsclc.nsclc_dataset import NSCLCDataset
importlib.reload(my_modules.custom_models.classifier_models)
from my_modules.custom_models.classifier_models import *

In [ ]:
# Load dataset
data = NSCLCDataset('E:\\NSCLC Data - PMD', ['orr', 'taumean', 'boundfraction'], label='M', mask_on=False)
data.normalize_channels_to_max()
data.augment()
data.show_random()

Cache reset


# Metastatic Potential Classification
## Image-based, custom classifier models
### Pre-trained performance evaluation
The following tests are performed on models trained over 125 epochs with a learning rate of 0.01 on augmented, normalized, and unmasked data composed of orr, mean lifetime, and bound fraction images.

In [ ]:
models = {'MLP': 
              [MLPNet, 'MLP Net_Epochs-125_nsclc_Metastases_orr+taumean+boundfraction_Augmented_Normalized.pth'],
          'Parallel MLP': 
              [ParallelMLPNet, 'Parallel MLP_Epochs-125_nsclc_Metastases_orr+taumean+boundfraction_Augmented_Normalized.pth'],
          'Regularized MLP': 
              [RegularizedMLPNet, 'Regularized MLP Net_Epochs-125_nsclc_Metastases_orr+taumean+boundfraction_Augmented_Normalized.pth'],
          'Regularized Parallel MLP': 
              [RegularizedParallelMLPNet, 'Regularized Parallel MLP Net_Epochs-125_nsclc_Metastases_orr+taumean+boundfraction_Augmented_Normalized.pth'],
          'CNN':
              [CNNet, 'CN Net_Epochs-125_nsclc_Metastases_orr+taumean+boundfraction_Augmented_Normalized.pth'],
          'Parallel CNN': 
              [ParallelCNNet, 'Parallel CN Net_Epochs-125_nsclc_Metastases_orr+taumean+boundfraction_Augmented_Normalized.pth'],
          'Regularized Parallel CNN': 
              [RegularizedParallelCNNet, 'Regularized Parallel CN Net_Epochs-125_nsclc_Metastases_orr+taumean+boundfraction_Augmented_Normalized.pth']
          }

In [ ]:
for model_name, (model_fn, model_path) in models.items():
    # Testing on dataset
    model = model_fn(data.shape)
    model.load_state_dict(
        torch.load(f'models/{model_path}',  map_location=torch.device('cpu')))
    
    model.eval()
    correct = 0
    data_out = []
    data_pred = []
    with torch.no_grad():
        for sample in data:
            out = model(sample[0].unsqueeze(0)).item()
            data_out.append(out)
            pred = np.round(out)
            data_pred.append(pred)
            correct += 1 if pred == sample[1].item() else 0
    print(f'Accuracy of {model_name} over whole dataset: {100*correct / len(data):.2f}%')

### Trash in $\rightarrow$ Trash out
As a sanity check, we have pushed through a "random" trash dataset below. We expect that trash in should give trash out. If it does, that is not a strong indication of model efficacy -- that will be addressed next. If it doesn't however, that is a strong indication of model failure.

Trash out we expect to look like a roughly random chance prediction, meaning about half of the predictions will be in either class. If this number starts to shift too high or low for the class, that means the model defaults to a certain class and may be only guessing once class most of the time. Fortunately, our data are failry well balanced (45/55% split), so if this happens on the actual dataset, the performance is limited to 55%. 

In [ ]:
def trash_in(model, trash_heap_size=500):
    _in = torch.rand(trash_heap_size, *model.input_size)
    trash_out = []
    model.eval()
    for trash in _in:
        _out = model(trash.unsqueeze(0)).item()
        trash_out.append(_out)
    return trash_out

In [ ]:
for model_name, (model_fn, model_path) in models.items():
    model = model_fn(data.shape)
    model.load_state_dict(torch.load(f'models/{model_path}',  map_location=torch.device('cpu')))
    trash_out = trash_in(model, 500)
    print(f'Trash into {model_name} resulted in {100*sum(trash_out) / len(trash_out):.2f}% of outputs being positively classified')

### 5-Fold Cross Validation
To assess efficacy, We will take the best performing models from this group (parallel MLPs and parallel CNNs) and perform 5-Fold Cross Validation to obtain more robust measures of performance for these models. For each fold, we will score raw accuracy and ROC-AUC.

#### Functions
Defining some useful functions so we can limit code copying

In [ ]:
from my_modules.model_learning import test_model
from my_modules.model_learning import fold_cross_validate
from my_modules.model_learning.model_phases import trash_in

# Creates folds by random sampling then multiplying samples by dataset numbers to augment without repeating slides in any group
from my_modules.model_learning.loader_maker import fold_augmented_data

Now we can use those functions to easily test our best-performers.

In [ ]:
best_models = [RegularizedParallelMLPNet, RegularizedParallelCNNet]
data_folds = fold_augmented_data(data)
batch_size = 32
learning_rate = 0.01
loss_fn = nn.BCELoss()
optimizer_fn = torch.optim.SGD
for model_fn in best_models:
    accuracy, running_loss, _  = fold_cross_validate(model_fn, data_folds, epochs=125, learning_rate=0.01)

We can also go ahead and run this test again with the masks on. If any of these models perform notably well, we can select them out and run 5-Fold Cross Validation across multiple hyperparameter sets to really fine tune things.

In [ ]:
data.mask_on = True
for model_fn in best_models:
    accuracy, running_loss, _  = fold_cross_validate(model_fn, data_folds, epochs=125, learning_rate=0.01)

## Histogram-based custom classifier models
First, we need to transform the dataset to match how the upcoming modes were trained. We can also take a look at some random samples to get an idea of what the model sees.

In [ ]:
data.dist_transform(nbins=25)
data.mask_on = False
data.show_random()

### Testing Pre-trained Models
These models are from a training session on the HPC. They were trained over varius hyperparameters and on the GPU. Some models were clearly better than others; accordingly, we already selected out the best performing architectures and are now comparing those models over a the gamut of hyperparams. 

In [ ]:
models = {'Fast MLP': 
              [MLPNet, 'MLP Net_Epochs-125_nsclc_Metastases_orr+taumean+boundfraction_Transformed_Augmented_Normalized.pth'],
          'Medium MLP': 
              [MLPNet, 'MLP Net_Epochs-250_nsclc_Metastases_orr+taumean+boundfraction_Transformed_Augmented_Normalized.pth'],
          'Slow MLP': 
              [MLPNet, 'MLP Net_Epochs-500_nsclc_Metastases_orr+taumean+boundfraction_Transformed_Augmented_Normalized.pth'],
          'Fast RNN':
              [RNNet, 'RN Net_Epochs-125_nsclc_Metastases_orr+taumean+boundfraction_Transformed_Augmented_Normalized.pth'],
          'Medium RNN': 
              [RNNet, 'RN Net_Epochs-250_nsclc_Metastases_orr+taumean+boundfraction_Transformed_Augmented_Normalized.pth'],
          'Slow RNN': 
              [RNNet, 'RN Net_Epochs-500_nsclc_Metastases_orr+taumean+boundfraction_Transformed_Augmented_Normalized.pth'],
          }

In [ ]:
for model_name, (model_fn, model_path) in models.items():
    # Testing on dataset
    model = model_fn(data.shape)
    model.load_state_dict(
        torch.load(f'models/{model_path}', map_location=torch.device('cpu')))
    
    model.eval()
    correct = 0
    data_out = []
    data_pred = []
    with torch.no_grad():
        for sample in data:
            out = model(sample[0].unsqueeze(0)).item()
            data_out.append(out)
            pred = np.round(out)
            data_pred.append(pred)
            correct += 1 if pred == sample[1].item() else 0
    print(f'Accuracy of {model_name} over whole dataset: {100*correct / len(data):.2f}%')

#### Trash in $\rightarrow$ Trash out Test

In [ ]:
for model_name, (model_fn, model_path) in models.items():
    model = model_fn(data.shape)
    model.load_state_dict(torch.load(f'models/{model_path}',  map_location=torch.device('cpu')))
    trash_out = trash_in(model, 500)
    print(f'Trash into {model_name} resulted in {100*sum(trash_out) / len(trash_out):.2f}% of outputs being positivley classified')

### 5-Fold Cross Validation 
First, we will use the basic and fast hyperparameters of 125 epochs with a learning rate of 0.01. 

In [ ]:
best_models = [MLPNet, ParallelMLPNet, RNNet]
lengths = [int(0.2 * len(data)) for _ in range(5)]
data_folds = torch.utils.data.random_split(dataset=data, lengths=lengths)

In [ ]:
for model_fn in best_models:
    accuracy, running_loss, _ = fold_cross_validate(model_fn, data_folds, epochs=125, learning_rate=0.01)

Next, we will iterate through a gamut of hyperparameters to try to find the best combination. Becasue the histogrma transformation saves so much model-complexity, we can afford to do this for all models out of the gate.

In [ ]:
# Try different hyperparameters in 5-Fold
epochs = [125, 250, 500, 1000, 2500]
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
for ep in epochs:
    for lr in learning_rates:
        print(f'\nEpochs: {ep} -- Learning Rate: {lr}\n______________________________________')
        for model_fn in best_models:
            accuracy, running_loss, _ = fold_cross_validate(model_fn, data_folds, epochs=ep, learning_rate=lr)
            print('______________________________________')

Finally, we will repeat this hyperparemeter iteration for the dataset with masks back on.

In [ ]:
data.mask_on = True
for ep in epochs:
    for lr in learning_rates:
        print(f'\nEpochs: {ep} -- Learning Rate: {lr}\n______________________________________')
        for model_fn in best_models:
            accuracy, running_loss, _ = fold_cross_validate(model_fn, data_folds, epochs=ep, learning_rate=lr)
            print('______________________________________')

# InceptionResNev2 + regularized MLP Classifier
This will use the exact specification fo the model architecture form the included source. In short, InceptionResNetV2 is used to extract feature maps which are then averaged to obtain asort of "feature spectrum." This is a nice way to add some depth to our model without adding a necessity for more data. InceptionResNetV2 is very deep (164 layers and 15 million parameers, I believe), but we aren't training it at all, so we don't have to worry about overfitting. We will just use the feature maps to train an MLP with 2 hidden layers each with dropouts to classify the liklihood of metastasis. This MP works very simililary to my Regualrized MLP Nets, but instead of outputting a binary class () or 1) it outputs a two-class liklihood (y, 1-(y)). To train, we will have to expand the original data labels to match this output, but functionally, it shouldn't change the question the model is trying to answer,. In fact this gives us better granualirty, as we will see driectly how certaiin the model is about the class prediciton.

> Schiele S, Arndt TT, Martin B, et al. Deep Learning Prediction of Metastasis in Locally Advanced Colon Cancer Using Binary Histologic Tumor Images. Cancers (Basel). 2021;13(9):2074. Published 2021 Apr 25. doi:10.3390/cancers13092074

In [ ]:
from pretrainedmodels import inceptionresnetv2
# NOTE: The automatica weight download for InceptionResNetV2 is broken. Model must be manually downloaded from https://data.lip6.fr/cadene/pretrainedmodels/inceptionresnetv2-520b38e4.pth and placed in C:/Users/<username>/.cache/torch/hub/checkpoints/
# Setup the dataset for this model
# Images, no mask (feature extractor will hopefully handle this), normalized (already is), 
data.mask_on = False
data.dist_transformed = False

batch_size = 21
learning_rate = 0.0005
optim_fn = torch.optim.RMSprop
epochs = 300
loss_fn = torch.nn.CrossEntropyLoss()

# Define our base feature extractor and turn the gradients off -- we won't train it, just use it to feed our MLP.
feature_extractor = inceptionresnetv2(num_classes=1000, pretrained='imagenet') # outputs 1536 feature maps when called with .features(x)
for params in feature_extractor.parameters():
    params.requires_grad = False


# Dry run feature extractor to get output dims for classifier creation
feature_extractor.eval()
with torch.no_grad():
    x = torch.rand(batch_size, data.shape[0], data.shape[1], data.shape[2])
    dry_run = feature_extractor.features(x)
feature_map_dims = dry_run.shape

In [ ]:
# Create classifier
class classifier(nn.Module):
    def __init__(self, input_size, feature_extractor):
        super(classifier, self).__init__()
        self.feature_extractor = feature_extractor
        self.input_size = input_size
        self.feature_map_dims = self.get_features(torch.rand(1, *input_size))
        
        self.GlobalAvgPool = nn.AvgPool2d(feature_map_dims[-2::], stride=2) # Get average value for each feature map (1536 feature values/image)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(1536, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 2)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.2)
        
    def forward(self, x):
        # Extract features
        x = self.get_features(x)
        
        # Classify
        x = self.GlobalAvgPool(x)
        x = self.flat(x)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x
    
    def get_features(self, x):
        return self.feature_extractor(x)

In [ ]:
# Check that shapes are as expected
m = classifier(x.shape[1:], feature_extractor.features)
out = m(x)
print(f'Input shape {x.shape}')
print(f'After feature extraction: {m.get_features(x).shape}')
print(f'After classifier: {out.shape}')

In [ ]:
models = []
accuracy = []
running_loss = []
for fold, test_set in enumerate(data_folds):
    # Make model, loaders, & optimizer for fold
    model = classifier(data.shape, feature_extractor.features)
    train_sets = [data_folds[index] for index in range(5) if index != fold]
    train_set = torch.utils.data.ConcatDataset(train_sets)
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
    optimizer = optimizer_fn(model.parameters(), lr=learning_rate)
    
    # Train
    model.train()
    for ep in range(epochs):
        print(f'Epoch: {ep}')
        for x, target in train_loader:
            # Extract features
            out = model(x)
            
            # Array to reform target to look like out
            y = torch.zeros_like(out) 
            
            # Put 100% certainty on the class
            for r, t in enumerate(target):
                y[r, t] = 1
            
            loss = loss_fn(out, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss.append(loss.item())
    # Test
    correct = 0
    for x, target in test_loader:
        out = model(x)
        pred = torch.argmax(out)
        correct += 1 if pred == target.item() else 0
        accuracy.append(100*correct/len(test_loader.sampler))
        models.append(model) 
        print(f'Inception + MLP accuracy for fold {fold+1}: {accuracy[-1]:.2f}%')
print(f'>>>Inception + MLP average accuracy: {sum(accuracy)/len(accuracy):.2f}%<<<')

# Manual Feature Extraction and Revision
Using the best performing models from 5-Fold Cross Validation, we will do some rudimentary feature extraction to refine the models. Training for 1000 epochs with a learning rate of 0.01 on histogram data proved to be the best overall method, yielding the highest overall accuracy (MLP Net with accuracy of 87.82%) and yielding relatively high accuracies for all networks (all three in >80%). We will train a fresh MLP Net with those settings and then average the 5 models to use for further analysis.

In [ ]:
data.dist_transform()
data.mask_on = False
accuracy, running_loss, models = fold_cross_validate(MLPNet, data_folds, epochs=10, learning_rate=0.01)

In [ ]:
from torchinfo import summary
summary(models[0], input_size=(batch_size,)+data.shape)

In [ ]:
# Get all params for each of the 5 models
sd = [m.state_dict() for m in models]
final_sd = {}

# Average all parameters into final state dict
for key in sd[0]:
    final_sd[key] = (sd[0][key] + sd[1][key] + sd[2][key] + sd[3][key] + sd[4][key]) / 5

# Create final model and run one final test on entire dataset
final_model = MLPNet(data.shape)
final_model.load_state_dict(final_sd)
all_data_loader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=True)
correct = test_model(final_model, all_data_loader)
print(f'Accuracy on whole dataset: {100* correct / len(all_data_loader.sampler):0.2f}%')

This accuracy is much worse than the accuracy of any of the individual models over the entire dataset. This is a good indication that that the models are not neccessarily reaching a global minimum in the loss function. That's not a terrible thing, but it does mean that the model needs to be generalized carefully. To this end, we are going to test an ensemble of models (since its so stinking light weight) to see if that is better. Then, we will proceed with feature extraction. But before any of that, look at the performance of the models on the entire dataset:

In [ ]:
for model in models:
    correct = test_model(model, all_data_loader)
    print(f'Accuracy on whole dataset: {100* correct / len(all_data_loader.sampler):0.2f}%')

In [ ]:
# For now, just take the best model of the 5
import matplotlib.pyplot as plt
model = models[-1]
sd = model.state_dict()
weight_mat = sd['fc1.weight'].T
plt.matshow(weight_mat, cmap=plt.cm.Accent)
plt.show()

weight_var = torch.var(weight_mat, axis=1)
fig2 = plt.figure(figsize=(17, 4.5))
plt.plot(weight_var)
plt.show()
print('test')

We can now use the variance of weights for any single input node to determine (roughly) which are actually doing something dynamic and which are basically just bias values. The one with more variance are actually affecting relationships. The ones that have a low variance are just "blanket" a near-single value into all the next nodes and may be effectively learned by the bias instead. We can find the input nodes that are just blanket connected (e.g. nodes 30-50) and remove those. We can then retrain and see how much of a difference it makes. Our accuracy may drop, since we will be passing less data, but the model can be made lighter. If the input nodes were really not meaningful, our accuracy won't change meaningfully. This will help us find what is more important to the model.

In [ ]:
# The fraction of total input nodes we will keep
fraction_to_take = 0.75
num_to_take = int(len(weight_var) * fraction_to_take)

# Create a list that pairs each weight variance with its original index so we can trace back after ranking
val_idx = list(enumerate(weight_var))

# Sort the pairs based on the value in descending order 
sorted_pairs = sorted(val_idx, key=lambda x: x[1], reverse=True)

# Create a dict that pairs where the value came from (index) with where it went (rank)
rank_dict = {index: rank for rank, (index, value) in enumerate(sorted_pairs)}

# Resort the ranks back into the matched locations using the original index
weight_rank = [rank_dict[i] for i in range(len(weight_var))]

# Look at the plot of just weights that made the cut
sel_weights = np.array([weight for (weight, rank) in zip(weight_rank, weight_rank) if rank < num_to_take])
fig = plt.matshow(sel_weights.T, cmap=plt.cm.Accent)
plt.show()

sel_weight_var = np.var(sel_weights.T, axis=0)
fig2 = plt.figure(figsize=(17, 4.5))
plt.plot(sel_weight_var)
plt.show()

In [ ]:
# Let's also list the weights by rank to make sense of what values are important
# First, we need to create a list of bin bounds based on the mode and nbins
bin_width = 1 / 25
bins = [[edge, edge + bin_width] for edge in np.arange(0, 1, bin_width)]

# Then we rescale each value back up so we have individual bin values for each mode
bin_values = [[[bin_value[0] * scalar, bin_value[1]* scalar] for bin_value in bins] for scalar in data.scalar]

# Finally, make a label for the actual bin range for each respective mode
bin_labels = [f'{mode} Bin: {low:0.2f} - {high:0.2f}' for mode, edge in zip(data.mode, bin_values) for low, high in edge]

# Then we need to sort them based on the sorting of the weight variances of their respective bins
sorted_labels = [bin_labels[rank] for rank in weight_rank]

# Now we can show this
for rank, label in enumerate(sorted_labels):
    print(f'#{rank}: {label}')

In [ ]:
# Retrain on selected features and test


In [ ]:
# Create ensemble of models
correct = 0
for x, target in all_data_loader:
    out = torch.mean(torch.stack([m(x) for m in models]), dim=0) # Get individual model outputs and then average them 
    pred = torch.round(out) # Get final prediction from rounding
    correct += torch.sum(pred.squeeze() == target).item()
print(f'Ensemble accuracy over all data set: {100 * correct / len(all_data_loader.sampler):0.2f}%')

This is super good, so we need to do something to check it. Obviously this isn't very robust becasue we've seen all this data before, even though no one model has seen it all, any piece of data 4/5 models ahve trained on. So what we can do is hold out just a few samples from all training, then use those as a final test set for the ensemble.

In [ ]:
##### TODO: Handle the above #####

## Model with all modes

In [ ]:
# Prep hyperparams
epochs = 125
learning_rate = 0.01
batch_size = 32
optimizer_fn = torch.optim.SGD
loss_fn = nn.BCEWithLogitsLoss()

# Prep data
data.mode = ['all']
data.mask_on = False
data_folds = fold_augmented_data(data)
model_fn = MLPNet
accuracy, running_loss, models = fold_cross_validate(model_fn, data_folds, epochs=125, learning_rate=0.01)

# Off-the-shelf classifiers

In [ ]:
import torchvision
import torchvision.models as tvm
torchvision.models.list_models()

In [ ]:
models = [tvm.mobilenet_v3_small, tvm.squeezenet1_1, tvm.alexnet, tvm.GoogLeNet]

In [ ]:
for model_fn in models:
    # Testing on dataset
    model = model_fn()
    
    model.eval()
    correct = 0
    data_out = []
    data_pred = []
    with torch.no_grad():
        for sample in data:
            out = model(sample[0].unsqueeze(0))
            pred = torch.argmax(out).item()
            data_pred.append(pred)
            correct += 1 if pred == sample[1].item() else 0
    print(f'Accuracy of {type(model)} over whole dataset: {100*correct / len(data):.2f}%')

In [ ]:
for model_fn in models:
    accuracy = []
    running_loss = []
    for fold, test_set in enumerate(data_folds):
        model = model_fn()
        train_sets = [data_folds[index] for index in range(5) if index != fold]
        train_set = torch.utils.data.ConcatDataset(train_sets)
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
        optimizer = optimizer_fn(model.parameters(), lr=lr)
        
        model.train()
        if torch.cuda.is_available() and not next(model.parameters()).is_cuda:
            model.to(torch.device('cuda'))
        total_loss = []
        for epoch in range(125):
            model.train(True)
            for x, target in train_loader:
                out = model(x).squeeze()
                # Convert out from one-hot encoding
                y = out/torch.sum(out)
                y = out[target] / (1 - out[target])
                loss = loss_fn(y, target)
                total_loss += loss.item()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            running_loss.append(total_loss)
        model.eval()
        correct = 0
        for x, target in test_loader:
            out = model(x).squeeze()
            # Convert out from one-hot encoding
            pred = torch.argmax(out).item()
            correct += (pred == target).sum().item()
        accuracy.append(100*correct/len(test_loader.sampler))
        print(f'{model.name} accuracy for fold {fold+1}: {accuracy[-1]:.2f}%')
    print(f'{model.name} average accuracy: {sum(accuracy)/len(accuracy):.2f}%')
    plt.plot(running_loss)
    plt.show()

# Results Decision
While these results are poor, the training parameters for these models was not expected to be ideal at only 125 epochs and a high learning rate. From these models, we will choose the best performers of both MLPs (for both image and histograms), CNNs, and RNNs and retrain on an array of hyperparameters to find the optimized model training. From those, the best will be characterized for final results.

## Results Summary for Image-based Classifiers
While the MLP-based networks showed some promising results (with the parallel MLPs reaching accuracies >80%), the parallel CNNs outperform all MLP-based networks and are of comparable weight to a fully-connected MLP through the use of kernels rather than dense layers. Interestingly, a basic CNN was the worst performer by a notable margin. This may be due to the increased level of complexity that the model must learn to detect relationships across dimensions, while not adding appropriate numbers of trainable parameters or increased data-size. Both parallel models performed excellently with accuracies >90%. The regularized parallel network only slightly outperforms the parallel network, but adds no new trainable parameters (and only 9KB to the model size) and should make performance generally more stable.

These models were all trained, evaluated, and tested originally on subsets of the entire dataset, but the tests shown here are across the entire dataset, meaning overfitting would be strongly reflected in the results. More robust checks for the best performers are below.